In [1]:
# Basic
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
from joblib import dump, load
import pickle

# GLM
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.families import family
from statsmodels.stats.multitest import multipletests
 
# Modelling
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge, Lasso, RidgeCV, LassoCV, ElasticNet, LogisticRegression
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier, StackingRegressor, StackingClassifier
from xgboost import XGBRegressor, XGBClassifier
import xgboost
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from mlxtend.regressor import StackingCVRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# SHAP
import shap

# Mertrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, confusion_matrix, precision_score, roc_curve, recall_score, precision_recall_curve, precision_recall_fscore_support, roc_auc_score, ConfusionMatrixDisplay, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score
from scipy.stats import randint
from sklearn.metrics import classification_report

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image

# import graphviz
import matplotlib.pyplot as plt
%matplotlib inline 

# glance wd
os.getcwd()

'/home/jhou2/HSV434/LandscapeProject/HSV434-IFNG-mechanism/Code'

In [2]:
os.chdir('/home/jhou2/HSV434/LandscapeProject/HSV434-IFNG-mechanism')

### Preprocessing data for ML: 
### Feature filtering based on significant gene, one-hot/Ordinal coding categorical features, scaling

In [3]:
# Load data and selected gene
selected_significant_genes = load('Processed/HSV434_Tcell_IFNG_mechanism_ML_Original_significant_genes')
exp_matrix_Original = load('Processed/HSV434_Tcell_IFNG_mechanism_exp_matrix')

In [4]:
# Select data based on significant_genes
if 'IFNG' not in selected_significant_genes['Gene']:
    gene_names = selected_significant_genes['Gene'].tolist()
    # assume CellType and Status may play roles here
    columns_to_select = ['CellType_Level3', 'Status', 'IFNG_bin']
    columns_to_select.extend(gene_names)

exp_matrix_Original_select = exp_matrix_Original[columns_to_select]
exp_matrix_Original_select.head(3)

,CellType_Level3,Status,IFNG_bin,CCL5,CCL4,CD69,TNF,IL7R,IL12RB2,HOPX,...,AKAP2,PTGER4,FXYD5,SLC9A3R1,CD160,SLAMF7,CSF1,EGR1,EOMES,TAGAP
Subject1_8WPH_AACTTTCCACTTAAGC-1,CD4 EM 2,Post,0,0.000000,0.0,0.000000,1.547185,3.637119,0.000000,2.127809,...,0.0,0.000000,3.291664,0.0,0.0,0.0,0.0,0.0,0.0,1.547185
Subject1_8WPH_AACTTTCTCAGCGATT-1,CD4 EM 3,Post,0,0.000000,0.0,0.000000,2.148939,0.000000,0.000000,0.000000,...,0.0,2.782014,2.148939,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Subject1_8WPH_ACAGCCGCATATACGC-1,CD4 TRM,Post,0,3.024739,0.0,3.693302,0.000000,0.000000,1.774471,1.774471,...,0.0,3.238121,1.774471,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [5]:
# Cell Type is nominal variable --> one-hot encoding
dummies_CellType = pd.get_dummies(exp_matrix_Original_select.CellType_Level3)
dummies_CellType = dummies_CellType.astype(int)
# remove Cell Type column
exp_matrix_Original_select = pd.concat([exp_matrix_Original_select.drop('CellType_Level3', axis=1), dummies_CellType], axis='columns')
exp_matrix_Original_select.head(3)

,Status,IFNG_bin,CCL5,CCL4,CD69,TNF,IL7R,IL12RB2,HOPX,JUN,...,CD4 EM 3,CD4 ISG,CD4 Prolif,CD4 TRM,CD8 CM,CD8 EM 1,CD8 EM 2,CD8 ISG,CD8 TRM 1,CD8 TRM 2
Subject1_8WPH_AACTTTCCACTTAAGC-1,Post,0,0.000000,0.0,0.000000,1.547185,3.637119,0.000000,2.127809,0.0,...,0,0,0,0,0,0,0,0,0,0
Subject1_8WPH_AACTTTCTCAGCGATT-1,Post,0,0.000000,0.0,0.000000,2.148939,0.000000,0.000000,0.000000,0.0,...,1,0,0,0,0,0,0,0,0,0
Subject1_8WPH_ACAGCCGCATATACGC-1,Post,0,3.024739,0.0,3.693302,0.000000,0.000000,1.774471,1.774471,0.0,...,0,0,0,1,0,0,0,0,0,0


In [6]:
# Status is Ordinal variable with inherent order --> Ordinal encoding
exp_matrix_Original_select['Status'].replace(['Prior', 'Lesion', 'Post'], [0, 1, 2], inplace=True)
exp_matrix_Original_select.head(2)

,Status,IFNG_bin,CCL5,CCL4,CD69,TNF,IL7R,IL12RB2,HOPX,JUN,...,CD4 EM 3,CD4 ISG,CD4 Prolif,CD4 TRM,CD8 CM,CD8 EM 1,CD8 EM 2,CD8 ISG,CD8 TRM 1,CD8 TRM 2
Subject1_8WPH_AACTTTCCACTTAAGC-1,2,0,0.0,0.0,0.0,1.547185,3.637119,0.0,2.127809,0.0,...,0,0,0,0,0,0,0,0,0,0
Subject1_8WPH_AACTTTCTCAGCGATT-1,2,0,0.0,0.0,0.0,2.148939,0.000000,0.0,0.000000,0.0,...,1,0,0,0,0,0,0,0,0,0


In [7]:
# prepare data X and y
X = exp_matrix_Original_select.drop(['IFNG_bin'], axis = 1)
X.head(3)

,Status,CCL5,CCL4,CD69,TNF,IL7R,IL12RB2,HOPX,JUN,GZMA,...,CD4 EM 3,CD4 ISG,CD4 Prolif,CD4 TRM,CD8 CM,CD8 EM 1,CD8 EM 2,CD8 ISG,CD8 TRM 1,CD8 TRM 2
Subject1_8WPH_AACTTTCCACTTAAGC-1,2,0.000000,0.0,0.000000,1.547185,3.637119,0.000000,2.127809,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
Subject1_8WPH_AACTTTCTCAGCGATT-1,2,0.000000,0.0,0.000000,2.148939,0.000000,0.000000,0.000000,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
Subject1_8WPH_ACAGCCGCATATACGC-1,2,3.024739,0.0,3.693302,0.000000,0.000000,1.774471,1.774471,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [8]:
y = exp_matrix_Original_select['IFNG_bin']
y.head(3)

Subject1_8WPH_AACTTTCCACTTAAGC-1    0
Subject1_8WPH_AACTTTCTCAGCGATT-1    0
Subject1_8WPH_ACAGCCGCATATACGC-1    0
Name: IFNG_bin, dtype: int64

In [9]:
print(f"X data shape:{X.shape}\n",
      f"y data shape:{y.shape}")

X data shape:(19143, 116)
 y data shape:(19143,)


In [10]:
# split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
print(f"Training data X: {X_train.shape}, y: {y_train.shape}\n",
      f"Testing data X: {X_test.shape}, y: {y_test.shape}")

Training data X: (13400, 116), y: (13400,)
 Testing data X: (5743, 116), y: (5743,)


In [11]:
# Apply scaling 
# Scaling: Fit only on the training set
# scaler = MinMaxScaler(feature_range=(0, 1))
scaler = StandardScaler()
X_train_scaled = X_train.copy()

columns_to_scale = X_train_scaled.columns.difference(['CD4 Act', 'CD4 CM', 'CD4 EM 1', 'CD4 EM 2', 'CD4 EM 3',
                                                      'CD4 ISG', 'CD4 Prolif', 'CD4 TRM', 'CD8 CM', 'CD8 EM 1', 'CD8 EM 2',
                                                      'CD8 ISG', 'CD8 TRM 1', 'CD8 TRM 2', 'Status'])
print(columns_to_scale)
X_train_scaled[columns_to_scale] = scaler.fit_transform(X_train_scaled[columns_to_scale])
X_train_scaled.head(3)

Index(['AC104078_2', 'ACOT7', 'ACTG1', 'ADGRG1', 'AKAP2', 'AL158071_4',
       'ALDOA', 'APOBEC3C', 'ATP8B4', 'BHLHE40',
       ...
       'SLC9A3R1', 'STOM', 'SVIL', 'TAGAP', 'TAP1', 'TNF', 'TNFSF9', 'TPM3',
       'TSC22D4', 'XCL1'],
      dtype='object', length=101)


,Status,CCL5,CCL4,CD69,TNF,IL7R,IL12RB2,HOPX,JUN,GZMA,...,CD4 EM 3,CD4 ISG,CD4 Prolif,CD4 TRM,CD8 CM,CD8 EM 1,CD8 EM 2,CD8 ISG,CD8 TRM 1,CD8 TRM 2
Subject10_Entry_ATGGGAGGTGAGGGAG-1,0,-1.230551,-0.411317,0.285152,-0.423639,-0.664458,-0.376361,0.292984,-0.130187,0.710791,...,0,0,0,1,0,0,0,0,0,0
Subject2_8WPH_GTACTCCGTCTGGTCG-1,2,0.894773,-0.411317,0.904245,-0.423639,0.203848,-0.376361,0.516764,-0.991971,0.949721,...,0,0,0,0,0,0,0,1,0,0
Subject17_Lesion_AGGTCCGGTAAGAGAG-1,1,-1.230551,-0.411317,-1.396325,-0.423639,0.324883,1.304481,-0.759225,-0.215809,-0.886076,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Apply the transformation to validation and test sets
X_test_scaled = X_test.copy()
X_test_scaled[columns_to_scale] = scaler.transform(X_test[columns_to_scale])
X_test_scaled.head(2)

,Status,CCL5,CCL4,CD69,TNF,IL7R,IL12RB2,HOPX,JUN,GZMA,...,CD4 EM 3,CD4 ISG,CD4 Prolif,CD4 TRM,CD8 CM,CD8 EM 1,CD8 EM 2,CD8 ISG,CD8 TRM 1,CD8 TRM 2
Subject5_Lesion_TCTGAGACATGGGACA-1,1,0.405401,-0.411317,1.496892,-0.423639,0.319752,3.199826,0.438151,1.133175,0.867285,...,0,0,0,0,0,0,0,0,0,0
Subject11_Lesion_ATTACTCCAGTTCCCT-1,1,0.646172,2.096039,0.267719,-0.423639,-0.291494,-0.376361,-0.759225,-0.991971,1.184535,...,0,0,0,0,0,1,0,0,0,0


### (1) Random Forest

In [13]:
# BayesianCV - search best parameters
n_iter = 100

rf_class_params = {'n_estimators': Integer(100, 1000),
                   'max_depth':Integer(1, 100),
                   'min_samples_split':Integer(2, 30)}

rf_class_grid = BayesSearchCV(estimator=RandomForestClassifier(),
                              search_spaces=rf_class_params,
                              n_iter=n_iter,
                              cv=5,
                              n_jobs=-1,
                              scoring='roc_auc',
                              random_state=123)

rf_class_grid.fit(X_train_scaled, y_train)

print("Best Random Forest Classifier Parameters:", rf_class_grid.best_params_)

Best Random Forest Classifier Parameters: OrderedDict([('max_depth', 14), ('min_samples_split', 2), ('n_estimators', 574)])


In [14]:
# Finalize the model by applying the best parameters
RF_class_best = RandomForestClassifier(max_depth=rf_class_grid.best_params_['max_depth'], 
                                       min_samples_split=rf_class_grid.best_params_['min_samples_split'],
                                       n_estimators=rf_class_grid.best_params_['n_estimators'])
RF_class_best.fit(X_train_scaled, y_train)

# Prediction on testing data
y_pred = RF_class_best.predict(X_test_scaled)

# Get predicted probabilities for ROC AUC
y_pred_proba = RF_class_best.predict_proba(X_test_scaled)[:, 1]  # Probability of the positive class (1)

# Evaluation
precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Precision: {precision:.4f}\n"
      f"Recall: {recall:.4f}\n"
      f"F1: {f1:.4f}\n"
      f"ROC_AUC: {roc_auc:.4f}")

Precision: 0.7111
Recall: 0.1157
F1: 0.1990
ROC_AUC: 0.7996


In [15]:
# The top-left: true negatives (TN).
# The top-right: false positives (FP).
# The bottom-left: false negatives (FN).
# The bottom-right: true positives (TP).
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[4874   39]
 [ 734   96]]


### (2) XGBoost

In [ ]:
# Bayesian CV - search best parameters
n_iter = 100

xgb_class_params = {'n_estimators': Integer(100, 1000),
                    'max_depth':Integer(1, 100),
                    'learning_rate':Real(0.0001, 0.1, prior='log-uniform')}

xgb_class_grid = BayesSearchCV(estimator = XGBClassifier(),
                               search_spaces = xgb_class_params,
                               n_iter = n_iter,
                               cv = 5,
                               n_jobs = -1,
                               scoring = 'roc_auc',
                               random_state = 123)

xgb_class_grid.fit(X_train_scaled, y_train)

print("Best XGB Classifier Parameters:", xgb_class_grid.best_params_)

In [22]:
# Finalize the model by appling the best parameters
xgb_class_best = XGBClassifier(max_depth = xgb_class_grid.best_params_['max_depth'],
                               learning_rate = xgb_class_grid.best_params_['learning_rate'],
                               n_estimators = xgb_class_grid.best_params_['n_estimators'])
xgb_class_best.fit(X_train_scaled, y_train)

# prediction on testing data
y_pred = xgb_class_best.predict(X_test_scaled)

# Get predicted probabilities for ROC AUC
y_pred_proba = xgb_class_best.predict_proba(X_test_scaled)[:, 1]  # Probability of the positive class (1)

# Evaluation
precision, recall, f, support = precision_recall_fscore_support(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Precision: {precision:.4f}\n"
      f"Recall: {recall:.4f}\n"
      f"F1: {f1:.4f}\n"
      f"ROC_AUC: {roc_auc:.4f}")

Precision: 0.6520
Recall: 0.2145
F1: 0.1990
ROC_AUC: 0.8053


### (3) SVM

In [23]:
# Bayesian CV - search best parameters
n_iter = 100
svc_class_param = {'C': Real(0.1, 1000, prior='log-uniform'),
                   'gamma':Real(0.0001, 1, prior='log-uniform')}

svc_class_grid = BayesSearchCV(estimator = SVC(kernel = 'rbf', probability=True),
                               search_spaces = svc_class_param,
                               n_iter = n_iter,
                               cv = 5,
                               n_jobs = -1,
                               scoring = 'roc_auc',
                               random_state = 123)
svc_class_grid.fit(X_train_scaled, y_train)
print("Best SVM Parameters:", svc_class_grid.best_params_)

Best SVM Parameters: OrderedDict([('C', 0.5063937940943523), ('gamma', 0.026601977931094496)])


In [24]:
# Finalize the model by appling the best parameters
svc_class_best = SVC(C = svc_class_grid.best_params_['C'], 
                     gamma = svc_class_grid.best_params_['gamma'], 
                     kernel = 'rbf', 
                     probability=True)
svc_class_best.fit(X_train_scaled, y_train)

# prediction on testing data
y_pred = svc_class_best.predict(X_test_scaled)

# Get predicted probabilities for ROC AUC
y_pred_proba = svc_class_best.predict_proba(X_test_scaled)[:, 1]  # Probability of the positive class (1)

# Evaluation
precision, recall, f, support = precision_recall_fscore_support(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Precision: {precision:.4f}\n"
      f"Recall: {recall:.4f}\n"
      f"F1: {f1:.4f}\n"
      f"ROC_AUC: {roc_auc:.4f}")

Precision: 0.0000
Recall: 0.0000
F1: 0.1990
ROC_AUC: 0.7836


/app/software/scikit-learn/1.3.1-gfbf-2023a/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### All three methods face the same issue that precision, and ROC_AUC is moderate, but Recall is very low. According to the formulas,  Precesion = TP/(TP + FP) and Recall = TP / (TP + FN).  The reason that Recall value is such low, it usually indicates the model is missing a lot of true positives, which suggests difficulty in distinguishing borderline cases or under-representation of certain sub-patterns in the positive class.